# Implement Kalman model using FastAI

> need to implement custom data preparation pipeline and loss function 

## Data Preparation

The aim of the data preparation pipeline is to:
- take the original time series and split it into time blocks
- for each block generate a random gap (need to figure out the properties of the gap)
- split some time blocks for testing

the input of the pipeline is:
- a dataframe containing all observations

the input of the model is:
- observed data (potentially containing NaN where data is missing)
- missing data mask (which is telling where the data is missing)
- the data needs to be standardized

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
#| default_exp kalman.fastai

In [ ]:
#| export
from meteo_imp.utils import *
from meteo_imp.gaussian import *

In [ ]:
reset_seed()

In [ ]:
import torch

In [ ]:
from fastai.tabular.core import *
from fastai.data.core import *

In [ ]:
#| export
from fastcore.transform import *
from fastcore.basics import *
from fastcore.foundation import *
from fastcore.all import *
from fastai.tabular import *
from fastai.torch_core import default_device

from meteo_imp.data import read_fluxnet_csv, hai_path

import collections

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
@cache_disk(cache_dir / "full_hai")
def load_data(dtype=np.float32):
    return read_fluxnet_csv(hai_path, None, num_dtype=dtype)

hai = load_data()
hai64 = load_data(np.float64)

### 1) Blocks

the first step is to transfrom the original dataframe into blocks of a specified `block_len`

two different strategies are possible:

- contigous blocks
- random block in the dataframe

In [ ]:
#| export
class BlockDfTransform(Transform):
    """divide timeseries DataFrame into blocks"""
    def __init__(self, df, block_len=200): 
        self.df = df 
        self.block_len = block_len
        self.n = len(df)
        
    def encodes(self, i:int) -> pd.DataFrame:       
        start = i * self.block_len
        end = (i+1) * self.block_len
        assert end <= self.n 
        
        block = self.df[start:end]
        
        return block

In [ ]:
blk = BlockDfTransform(hai, 10)

In [ ]:
blk

BlockDfTransform:
encodes: (int,object) -> encodes
decodes: 

In [ ]:
blk(1)

,TA,SW_IN,VPD
time,,,
2000-01-01 05:30:00,-0.23,0.00,0.138
2000-01-01 06:00:00,-0.23,0.00,0.122
2000-01-01 06:30:00,-0.22,0.00,0.098
2000-01-01 07:00:00,-0.24,0.00,0.066
2000-01-01 07:30:00,-0.23,0.00,0.044
2000-01-01 08:00:00,-0.22,0.00,0.026
2000-01-01 08:30:00,-0.19,0.45,0.016
2000-01-01 09:00:00,-0.14,3.70,0.010
2000-01-01 09:30:00,-0.03,7.26,0.006


In [ ]:
180 * 24 * 2 / 10

864.0

we are taking a day in the summer so there is an higher values for the variables

In [ ]:
blk(800)

,TA,SW_IN,VPD
time,,,
2000-06-15 16:30:00,14.65,468.190002,6.454
2000-06-15 17:00:00,14.22,224.800003,5.799
2000-06-15 17:30:00,14.11,195.279999,6.577
2000-06-15 18:00:00,14.23,244.169998,6.931
2000-06-15 18:30:00,14.40,253.919998,7.286
2000-06-15 19:00:00,14.09,177.309998,7.251
2000-06-15 19:30:00,13.71,97.070000,6.683
2000-06-15 20:00:00,13.08,39.709999,5.851
2000-06-15 20:30:00,12.41,10.650000,5.254


In [ ]:
tfms1 = TfmdLists([800,801,802,803], [BlockDfTransform(hai, 10)])

In [ ]:
tfms1[0]

,TA,SW_IN,VPD
time,,,
2000-06-15 16:30:00,14.65,468.190002,6.454
2000-06-15 17:00:00,14.22,224.800003,5.799
2000-06-15 17:30:00,14.11,195.279999,6.577
2000-06-15 18:00:00,14.23,244.169998,6.931
2000-06-15 18:30:00,14.40,253.919998,7.286
2000-06-15 19:00:00,14.09,177.309998,7.251
2000-06-15 19:30:00,13.71,97.070000,6.683
2000-06-15 20:00:00,13.08,39.709999,5.851
2000-06-15 20:30:00,12.41,10.650000,5.254


### 2) Gaps

adds a mask which includes a random gap

In [ ]:
class MaskedDf:
    def __init__(self,*args):
        self.data = args[0]
        self.mask = args[1]
    def __iter__(self): return iter((self.data, self.mask,))
    __repr__ = basic_repr("data, mask")
    def _repr_html_(self):
        return row_dfs({'data': self.data, 'mask': self.mask}, title="Masked Df")

In [ ]:
#| exports
def _make_random_gap(
    gap_length: int, # The length of the gap
    total_length: int, # The total number of observations
    gap_start: int = None # Optional start of gap
): # (total_length) array of bools to indicicate if the data is missing or not
    "Add a continous gap of ginve length at random position"
    if(gap_length >= total_length):
        return np.repeat(True, total_length)
    gap_start = np.random.randint(total_length - gap_length) if gap_start is None else gap_start
    return np.hstack([
        np.repeat(False, gap_start),
        np.repeat(True, gap_length),
        np.repeat(False, total_length - (gap_length + gap_start))
    ])

In [ ]:
#| export
from fastcore.basics import *

In [ ]:
#| export
class AddGapTransform(Transform):
    """Adds a random gap to a `TimeSTensor`"""
    def __init__(self,
                variables,
                gap_length,
                ):
        store_attr()
    def encodes(self, df: pd.DataFrame):
        gap = _make_random_gap(self.gap_length, df.shape[0])
        mask = np.ones_like(df, dtype=bool)
        col_sel = L(*df.columns).argwhere(lambda x: x in self.variables)
        mask[np.argwhere(gap), col_sel] = False
        return MaskedDf(df, pd.DataFrame(mask, index=df.index, columns=df.columns))

In [ ]:
a_gap = AddGapTransform(['TA', 'VPD'], 5)
a_gap

AddGapTransform:
encodes: (DataFrame,object) -> encodes
decodes: 

In [ ]:
a_gap(blk(800))

TA,SW_IN,VPD
14.6500,468.1900,6.4540
14.2200,224.8000,5.7990
14.1100,195.2800,6.5770
14.2300,244.1700,6.9310
14.4000,253.9200,7.2860
14.0900,177.3100,7.2510
13.7100,97.0700,6.6830
13.0800,39.7100,5.8510
12.4100,10.6500,5.2540
12.2700,0.3200,5.1640


In [ ]:
m_df = a_gap(blk(800))

In [ ]:
display_as_row({'data': m_df.data, 'mask': m_df.mask})

TA,SW_IN,VPD
14.6500,468.1900,6.4540
14.2200,224.8000,5.7990
14.1100,195.2800,6.5770
14.2300,244.1700,6.9310
14.4000,253.9200,7.2860
14.0900,177.3100,7.2510
13.7100,97.0700,6.6830
13.0800,39.7100,5.8510
12.4100,10.6500,5.2540
12.2700,0.3200,5.1640


In [ ]:
tfms2 = TfmdLists([800,801,802,803], [BlockDfTransform(hai, 10), AddGapTransform(['TA','SW_IN'], 2)])

In [ ]:
tfms2[0]

TA,SW_IN,VPD
14.6500,468.1900,6.4540
14.2200,224.8000,5.7990
14.1100,195.2800,6.5770
14.2300,244.1700,6.9310
14.4000,253.9200,7.2860
14.0900,177.3100,7.2510
13.7100,97.0700,6.6830
13.0800,39.7100,5.8510
12.4100,10.6500,5.2540
12.2700,0.3200,5.1640


In [ ]:
#| export
@patch
def tidy(self: MaskedDf):
    data = self.data.reset_index().melt("time")
    mask = self.mask.reset_index().melt("time", value_name="is_present")
    
    return pd.merge(data, mask, on=["time", "variable"])

In [ ]:
m_df.tidy()

,time,variable,value,is_present
0,2000-06-15 16:30:00,TA,14.650000,False
1,2000-06-15 17:00:00,TA,14.220000,False
2,2000-06-15 17:30:00,TA,14.110000,False
3,2000-06-15 18:00:00,TA,14.230000,False
4,2000-06-15 18:30:00,TA,14.400000,False
5,2000-06-15 19:00:00,TA,14.090000,True
6,2000-06-15 19:30:00,TA,13.710000,True
7,2000-06-15 20:00:00,TA,13.080000,True
8,2000-06-15 20:30:00,TA,12.410000,True
9,2000-06-15 21:00:00,TA,12.270000,True


#### Plotting

In [ ]:
#| export
import altair as alt
from altair import datum

In [ ]:
#| exporti
def def_selection():
    return alt.selection_interval(bind="scales")

In [ ]:
#| exporti
def plot_rug(df, sel = def_selection(), props = {}):
    if 'height' in props:
        props = props.copy() 
        props.pop('height') # rug should have default heigth
    return alt.Chart(df).mark_tick(
            color='black',
        ).encode(
            x = "time",
            color = alt.condition(datum.is_present, alt.value('white'), alt.value('black'))
        ).add_selection(
            sel
        ).properties(**props) 

In [ ]:
plot_rug(m_df.tidy())

/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning)


alt.Chart(...)

In [ ]:
def plot_points(df, y_label = "", sel = def_selection(), props = {}):
    return alt.Chart(df).mark_point(
            color='black',
            strokeWidth = 1,
            fillOpacity = 1
        ).encode(
            x = alt.X("time", axis=alt.Axis(domain=False, labels = False, ticks=False, title=None)),
            y = alt.Y("value", title = y_label, scale=alt.Scale(zero=False)),
            fill= alt.Fill("is_present", scale = alt.Scale(range=["black", "#ffffff00"]),
                           legend = alt.Legend(title =["Observed data"])),
            shape = "is_present",
        )

In [ ]:
plot_points(m_df.tidy())

alt.Chart(...)

In [ ]:
#| exporti
def plot_line(df, only_present=True, y_label = "", sel = def_selection(), props = {}):
    # df = df[df.is_present] if only_present else df
    # TODO remove onle_present
    return alt.Chart(df).mark_line().encode(
        x = "time",    
        y = alt.Y("value", title = y_label, scale=alt.Scale(zero=False)),
        color='variable'
    ).add_selection(
        sel
    ).properties(
        **props
    )#.transform_filter(
    #     datum.is_present
    # )

    

In [ ]:
plot_line(m_df.tidy())

/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning)


alt.Chart(...)

In [ ]:
#| exporti
def plot_variable(df, variable, title="", y_label="", sel = None, props = {}):
    df = df[df.variable == variable]
    sel = ifnone(sel, def_selection())
    # rug = plot_rug(df, sel, props)
    points = plot_points(df, y_label, sel, props)
    line = plot_line(df, True, y_label, sel, props)
    
    return (points + line).properties(title=title)
    
    # return alt.VConcatChart(vconcat=[(points + line), rug], spacing=-10).properties(title=title)

In [ ]:
plot_variable(m_df.tidy(), "TA", title="title TA")

/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning)


alt.LayerChart(...)

In [ ]:
#| export
@patch
def show(self: MaskedDf, ax=None, ctx=None, 
        n_cols: int = 3,
        bind_interaction: bool =True, # Whether the sub-plots for each variable should be connected for zooming/panning
        props:dict = None # additional properties (eg. size) for altair plot
       ) -> alt.Chart:
    
    df = self.tidy()
    
    props = ifnone(props, {'width': 180, 'height': 100})
   
    plot_list = [alt.hconcat() for _ in range(0, self.data.shape[0], n_cols)]
    selection_scale = alt.selection_interval(bind="scales", encodings=['x']) if bind_interaction else None
    for idx, variable in enumerate(self.data.columns):
        plot = plot_variable(df,
                            variable,
                            title = variable,
                            y_label = variable,
                            sel = selection_scale,
                            props=props)
        
        plot_list[idx // n_cols] |= plot
    
    plot = alt.vconcat(*plot_list)
    
    return plot

In [ ]:
m_df.show()

/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning)


alt.VConcatChart(...)

In [ ]:
a_gap(blk(799)).show()

/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning)


alt.VConcatChart(...)

In [ ]:
idx = L(*blk(1).columns).argwhere(lambda x: x in ['TA','SW_IN'])

In [ ]:
mask = np.ones_like(blk(1), dtype=bool)

In [ ]:
mask

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [ ]:
gap = _make_random_gap(2, 10, 2)

In [ ]:
gap

array([False, False,  True,  True, False, False, False, False, False,
       False])

In [ ]:
np.argwhere(gap)

array([[2],
       [3]])

In [ ]:
mask[np.argwhere(gap), idx] = False

In [ ]:
mask

array([[ True,  True,  True],
       [ True,  True,  True],
       [False, False,  True],
       [False, False,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [ ]:
mask[gap]

array([[False, False,  True],
       [False, False,  True]])

### To Tensor

this needs to handle both the init with a list of items and when the first item is a sequence of list of items

In [ ]:
class MaskedTensor(collections.abc.Sequence):
    def __init__(self,*args):
        if len(args)==2:
            self.data = args[0]
            self.mask = args[1]
        elif len(args)==1 and len(args[0])==2:
            self.data = args[0][0]
            self.mask = args[0][1]
        else:
            raise ValueError(f"Incorrect number of arguments. got {len(args)} args")

    def __iter__(self): return iter((self.data, self.mask,))
    __len__ = 2
    def __getitem__(self, key):
        if key == 0: return self.data
        elif key == 1: return self.mask
        else: raise IndexError("index bigger than 2")
    __repr__ = basic_repr('data, mask')

In [ ]:
#| export
class MaskedDf2Tensor(Transform):
    def setups(self, items):
        self.columns = list(items[0].data.columns)
    def encodes(self, df: MaskedDf) -> MaskedTensor:
        data = torch.tensor(df.data.to_numpy())
        mask = torch.tensor(df.mask.to_numpy())
        return MaskedTensor(data, mask)
        
    def decodes(self, x: MaskedTensor) -> MaskedDf:
        data = pd.DataFrame(x.data.detach().cpu().numpy(), columns = self.columns)
        mask = pd.DataFrame(x.mask.cpu().numpy(), columns = self.columns)
        return MaskedDf(data, mask)

In [ ]:
to_t = MaskedDf2Tensor()

In [ ]:
to_t.setup(tfms2)

In [ ]:
to_t(tfms2[0])

__main__.MaskedTensor(data=tensor([[1.4650e+01, 4.6819e+02, 6.4540e+00],
        [1.4220e+01, 2.2480e+02, 5.7990e+00],
        [1.4110e+01, 1.9528e+02, 6.5770e+00],
        [1.4230e+01, 2.4417e+02, 6.9310e+00],
        [1.4400e+01, 2.5392e+02, 7.2860e+00],
        [1.4090e+01, 1.7731e+02, 7.2510e+00],
        [1.3710e+01, 9.7070e+01, 6.6830e+00],
        [1.3080e+01, 3.9710e+01, 5.8510e+00],
        [1.2410e+01, 1.0650e+01, 5.2540e+00],
        [1.2270e+01, 3.2000e-01, 5.1640e+00]]), mask=tensor([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [False, False,  True],
        [False, False,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]))

In [ ]:
to_t.decode(to_t(tfms2[0]));

In [ ]:
tfms2[0]

TA,SW_IN,VPD
14.6500,468.1900,6.4540
14.2200,224.8000,5.7990
14.1100,195.2800,6.5770
14.2300,244.1700,6.9310
14.4000,253.9200,7.2860
14.0900,177.3100,7.2510
13.7100,97.0700,6.6830
13.0800,39.7100,5.8510
12.4100,10.6500,5.2540
12.2700,0.3200,5.1640


In [ ]:
type(MaskedDf2Tensor())

__main__.MaskedDf2Tensor

In [ ]:
tfms3 = TfmdLists([800, 801, 802], [BlockDfTransform(hai, 10), AddGapTransform(['TA','SW_IN'], 2), MaskedDf2Tensor()])

In [ ]:
tfms3[0]

__main__.MaskedTensor(data=tensor([[1.4650e+01, 4.6819e+02, 6.4540e+00],
        [1.4220e+01, 2.2480e+02, 5.7990e+00],
        [1.4110e+01, 1.9528e+02, 6.5770e+00],
        [1.4230e+01, 2.4417e+02, 6.9310e+00],
        [1.4400e+01, 2.5392e+02, 7.2860e+00],
        [1.4090e+01, 1.7731e+02, 7.2510e+00],
        [1.3710e+01, 9.7070e+01, 6.6830e+00],
        [1.3080e+01, 3.9710e+01, 5.8510e+00],
        [1.2410e+01, 1.0650e+01, 5.2540e+00],
        [1.2270e+01, 3.2000e-01, 5.1640e+00]]), mask=tensor([[ True,  True,  True],
        [ True,  True,  True],
        [False, False,  True],
        [False, False,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]))

In [ ]:
type(tfms3[0])

__main__.MaskedTensor

In [ ]:
tfms3.decode(tfms3[0])

TA,SW_IN,VPD
14.6500,468.1900,6.4540
14.2200,224.8000,5.7990
14.1100,195.2800,6.5770
14.2300,244.1700,6.9310
14.4000,253.9200,7.2860
14.0900,177.3100,7.2510
13.7100,97.0700,6.6830
13.0800,39.7100,5.8510
12.4100,10.6500,5.2540
12.2700,0.3200,5.1640


### Normalize

In [ ]:
#| export
from meteo_imp.utils import *
from fastai.torch_core import to_cpu

from torch import Tensor

In [ ]:
class NormalsParams(collections.abc.Sequence):
    def __init__(self,*args):
        self.mean = args[0]
        self.std = args[1]
    def __iter__(self): return iter((self.mean, self.std,))
    __len__ = 2
    def __getitem__(self, key):
        if key == 0: return self.mean
        elif key == 1: return self.std
        else: raise IndexError("index bigger than 2")
    __repr__ = basic_repr('mean, std')

In [ ]:
#| export
def get_stats(df, device='cpu'):
    return torch.tensor(df.mean(axis=0).to_numpy(), device=device), torch.tensor(df.std(axis=0).to_numpy(), device=device)

In [ ]:
#| export
class NormalizeMasked(Transform):
    "Normalize/denorm MaskedTensor column-wise "
    @property
    def name(self): return f"{super().name} -- {getattr(self,'__stored_args__',{})}"

    def __init__(self, mean=None, std=None): store_attr()

    def encodes(self, x:MaskedTensor)-> MaskedTensor:
        return MaskedTensor((x.data -self.mean) / self.std, x.mask)

    def decodes(self, x:MaskedTensor)->MaskedTensor:
        f = to_cpu if x[0].device.type=='cpu' else noop
        return MaskedTensor(x[0] * f(self.std) + f(self.mean), x[1])
    
    def decodes(self, x:NormalsParams):
        f = to_cpu if x[0].device.type=='cpu' else noop
        mean = x.mean * f(self.std) + f(self.mean)
        std = x.std * (self.std)
        
        return NormalsParams(mean, std)

In [ ]:
norm = NormalizeMasked(*get_stats(hai))

In [ ]:
tfms3[0]

__main__.MaskedTensor(data=tensor([[1.4650e+01, 4.6819e+02, 6.4540e+00],
        [1.4220e+01, 2.2480e+02, 5.7990e+00],
        [1.4110e+01, 1.9528e+02, 6.5770e+00],
        [1.4230e+01, 2.4417e+02, 6.9310e+00],
        [1.4400e+01, 2.5392e+02, 7.2860e+00],
        [1.4090e+01, 1.7731e+02, 7.2510e+00],
        [1.3710e+01, 9.7070e+01, 6.6830e+00],
        [1.3080e+01, 3.9710e+01, 5.8510e+00],
        [1.2410e+01, 1.0650e+01, 5.2540e+00],
        [1.2270e+01, 3.2000e-01, 5.1640e+00]]), mask=tensor([[ True,  True,  True],
        [ True,  True,  True],
        [False, False,  True],
        [False, False,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]))

In [ ]:
test_close(norm.decode(norm(tfms3[0]))[0], tfms3[0][0], eps=2e-5)

Test that NormalsParams decode actually works

In [ ]:
Npars = NormalsParams(torch.tensor(1), torch.tensor(.1))

In [ ]:
norm.decode(Npars)

__main__.NormalsParams(mean=tensor([ 16.2585, 324.9604,   7.7491]), std=tensor([ 0.7925, 20.4003,  0.4368]))

In [ ]:
tfms4 = TfmdLists([800,801,803], [BlockDfTransform(hai, 10), 
                           AddGapTransform(['TA','SW_IN'], 2),
                           MaskedDf2Tensor(),
                           NormalizeMasked(*get_stats(hai,device='cpu'), ) ])

In [ ]:
tfms4[0]

__main__.MaskedTensor(data=tensor([[ 0.7970,  1.7021,  0.7035],
        [ 0.7428,  0.5090,  0.5536],
        [ 0.7289,  0.3643,  0.7317],
        [ 0.7440,  0.6040,  0.8127],
        [ 0.7655,  0.6518,  0.8940],
        [ 0.7264,  0.2762,  0.8860],
        [ 0.6784, -0.1171,  0.7560],
        [ 0.5989, -0.3983,  0.5655],
        [ 0.5144, -0.5407,  0.4288],
        [ 0.4967, -0.5914,  0.4082]]), mask=tensor([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [False, False,  True],
        [False, False,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]))

In [ ]:
tfms4.decode(tfms4[0])

TA,SW_IN,VPD
14.6500,468.1900,6.4540
14.2200,224.8000,5.7990
14.1100,195.2800,6.5770
14.2300,244.1700,6.9310
14.4000,253.9200,7.2860
14.0900,177.3100,7.2510
13.7100,97.0700,6.6830
13.0800,39.7100,5.8510
12.4100,10.6500,5.2540
12.2700,0.3200,5.1640


is workinggggggggggggggggg 

###### Old Transform debug

In [ ]:
type(norm(tfms3[0]))

__main__.MaskedTensor

In [ ]:
t = TypeDispatch()

In [ ]:
def a(x: int) -> int:
    return a +1

In [ ]:
def b(x: float) -> int:
    return a +2

In [ ]:
t.add(a)

In [ ]:
t.add(b)

In [ ]:
t.funcs

{<class 'int'>: {<class 'object'>: <function a>}, <class 'float'>: {<class 'object'>: <function b>}}

In [ ]:
t

(int,object) -> a
(float,object) -> b

In [ ]:
norm.encodes.funcs

{<class '__main__.MaskedTensor'>: {<class 'object'>: <function NormalizeMasked.encodes>}}

In [ ]:
t[int]

<function __main__.a(x: int) -> int>

In [ ]:
norm.encodes[MaskedTensor]

<function __main__.NormalizeMasked.encodes(self, x: __main__.MaskedTensor) -> __main__.MaskedTensor>

In [ ]:
norm.decodes[MaskedTensor]

<function __main__.NormalizeMasked.decodes(self, x: __main__.MaskedTensor) -> __main__.MaskedTensor>

In [ ]:
norm.decodes[NormalsParams]

<function __main__.NormalizeMasked.decodes(self, x: __main__.NormalsParams)>

In [ ]:
x = tfms3[0]

In [ ]:
type(x)

__main__.MaskedTensor

In [ ]:
norm.decode(x)

__main__.MaskedTensor(data=tensor([[1.2443e+02, 9.5633e+04, 3.1574e+01],
        [1.2102e+02, 4.5981e+04, 2.8713e+01],
        [1.2015e+02, 3.9959e+04, 3.2112e+01],
        [1.2110e+02, 4.9932e+04, 3.3658e+01],
        [1.2245e+02, 5.1921e+04, 3.5209e+01],
        [1.1999e+02, 3.6293e+04, 3.5056e+01],
        [1.1698e+02, 1.9923e+04, 3.2575e+01],
        [1.1199e+02, 8.2219e+03, 2.8940e+01],
        [1.0668e+02, 2.2936e+03, 2.6332e+01],
        [1.0557e+02, 1.8624e+02, 2.5939e+01]]), mask=tensor([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [False, False,  True],
        [False, False,  True],
        [ True,  True,  True]]))

In [ ]:
y = NormalsParams(x[0], x[0])

In [ ]:
def _is_tuple(o): return isinstance(o, tuple) and not hasattr(o, '_fields')

In [ ]:
class ItemTransform(Transform):
    "A transform that always take tuples as items"
    _retain = True
    def __call__(self, x, **kwargs): return self._call1(x, '__call__', **kwargs)
    def decode(self, x, **kwargs):   return self._call1(x, 'decode', **kwargs)
    def _call1(self, x, name, **kwargs):
        if not _is_tuple(x): return getattr(super(), name)(x, **kwargs)
        y = getattr(super(), name)(list(x), **kwargs)
        if not self._retain: return y
        if is_listy(y) and not isinstance(y, tuple): y = tuple(y)
        return retain_type(y, x)
    def _call2(self, x, name, **kwargs):
        if not _is_tuple(x): return getattr(super(), name)
        return getattr(super(), name)
        if not self._retain: return y
        if is_listy(y) and not isinstance(y, tuple): y = tuple(y)
        return retain_type(y, x)

In [ ]:
basic_repr??

Signature: basic_repr(flds=None)
Source:   
def basic_repr(flds=None):
    "Minimal `__repr__`"
    if isinstance(flds, str): flds = re.split(', *', flds)
    flds = list(flds or [])
    def _f(self):
        res = f'{type(self).__module__}.{type(self).__name__}'
        if not flds: return f'<{res}>'
        sig = ', '.join(f'{o}={getattr(self,o)!r}' for o in flds)
        return f'{res}({sig})'
    return _f
File:      ~/.local/lib/python3.10/site-packages/fastcore/basics.py
Type:      function


In [ ]:
class myT(ItemTransform):
    def encodes(self, x: NormalsParams):
        return NormalsParams(0,0)
    def decodes(self, x: NormalsParams):
        return NormalsParams(1,1)

In [ ]:
t = myT()

In [ ]:
y = NormalsParams(0,1)

In [ ]:
y

__main__.NormalsParams(mean=0, std=1)

In [ ]:
t.decode(y)

__main__.NormalsParams(mean=1, std=1)

In [ ]:
f = t._call2(y, 'decode')

In [ ]:
t.decodes

(NormalsParams,object) -> decodes

In [ ]:
import inspect

In [ ]:
t(y)

__main__.NormalsParams(mean=0, std=0)

In [ ]:
type(y)

__main__.NormalsParams

In [ ]:
t.decodes[NormalsParams]

<function __main__.myT.decodes(self, x: __main__.NormalsParams)>

In [ ]:
t(x)

__main__.MaskedTensor(data=tensor([[1.4650e+01, 4.6819e+02, 6.4540e+00],
        [1.4220e+01, 2.2480e+02, 5.7990e+00],
        [1.4110e+01, 1.9528e+02, 6.5770e+00],
        [1.4230e+01, 2.4417e+02, 6.9310e+00],
        [1.4400e+01, 2.5392e+02, 7.2860e+00],
        [1.4090e+01, 1.7731e+02, 7.2510e+00],
        [1.3710e+01, 9.7070e+01, 6.6830e+00],
        [1.3080e+01, 3.9710e+01, 5.8510e+00],
        [1.2410e+01, 1.0650e+01, 5.2540e+00],
        [1.2270e+01, 3.2000e-01, 5.1640e+00]]), mask=tensor([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [False, False,  True],
        [False, False,  True],
        [ True,  True,  True]]))

In [ ]:
t(y)

__main__.NormalsParams(mean=0, std=0)

In [ ]:
type(y)

__main__.NormalsParams

In [ ]:
norm.decode(y)

AttributeError: 'int' object has no attribute 'device'

In [ ]:
type(norm.decode(y))

In [ ]:
norm.decodes

In [ ]:
tfms4 = TfmdLists([800,801,803], [BlockDfTransform(hai, 10), 
                           AddGapTransform(['TA','SW_IN'], 2),
                           MaskedDf2Tensor(),
                           NormalizeMasked(*get_stats(hai,device='cpu'), ) ])

In [ ]:
tfms4

In [ ]:
tfms4[0]

In [ ]:
tfms3[0][0].mean(axis=0), tfms3[0][0].std(axis=0)

In [ ]:
tfms4[0][0].mean(axis=0), tfms4[0][0].std(axis=0)

In [ ]:
tfms4.tfms.fs.__repr__??

In [ ]:
tfms4.tfms.fs.items??

In [ ]:
@patch
def __repr__(self: L): return '\n'.join([repr(o) for o in self.items])

In [ ]:
tfms4

In [ ]:
type(tfms4[0])

In [ ]:
norm = NormalizeMasked(*get_stats(hai))

In [ ]:
norm

In [ ]:
norm.encodes(tfms[0])

In [ ]:
b = TfmdLists([0,1], [BlockDfTransform(hai, 10), AddGapTransform(['TA','SW_IN'], 2), MaskedDf2Tensor, NormalizeMasked(*get_stats(hai))]).dataloaders(bs=2).one_batch()[0]

In [ ]:
b[0].mean(0)

In [ ]:
b[1].mean(0)

In [ ]:
b.std(axis=(0,1))

### Pipeline

In [ ]:
#| export
from fastai.data.transforms import *

In [ ]:
block_len = 10
block_ids = list(range(0, (len(hai) // block_len) - 1))[:10]
gap_len = 2

In [ ]:
#| export
def imp_pipeline(df,
                 block_len,
                 gap_len
                ):
    block_ids = list(range(0, (len(df) // block_len) - 1))
    return [BlockDfTransform(df, block_len),
            AddGapTransform(['TA','SW_IN'], gap_len),
            MaskedDf2Tensor,
            NormalizeMasked(*get_stats(df))], block_ids

In [ ]:
pipeline, block_ids = imp_pipeline(hai, block_len, gap_len)

In [ ]:
pipeline

In [ ]:
pp = Pipeline(pipeline)

In [ ]:
pp

### Dataloader

random splitter for validation/training set

In [ ]:
reset_seed()

In [ ]:
splits = RandomSplitter()(block_ids)

Repeat twice the pipeline since is the same pipeline both for training data and for labels

In [ ]:
import collections

In [ ]:
def to_tuple(x):
    return tuple(x)

In [ ]:
isinstance(tfms4[0], Sequence)

In [ ]:
ds = Datasets(block_ids, [pipeline, pipeline], splits=splits)

In [ ]:
from torch.utils.data._utils.collate import default_collate

In [ ]:
dls = ds.dataloaders(bs=1)

In [ ]:
dls.device

In [ ]:
dls.one_batch()

In [ ]:
@typedispatch
def show_batch(x: MaskedTensor, y, samples, ctxs=None, max_n=6):
    print(x)

In [ ]:
dls.show_batch()

In [ ]:
dls._types

In [ ]:
dls.show_batch()

In [ ]:
Datasets

In [ ]:
#| export
def make_dataloader(df, block_len, gap_len, bs=10):
    pipeline, block_ids = imp_pipeline(df, block_len, gap_len)
    
    splits = RandomSplitter()(block_ids)
    ds = Datasets(block_ids, [pipeline, pipeline], splits=splits)
    
    return ds.dataloaders(bs=bs)
    

In [ ]:
dls = make_dataloader(hai, 200, 10)

In [ ]:
dls.one_batch()[0][0].shape

In [ ]:
dls = dls.cpu()

## Model

### Forward Function

in order to the a pytorch module we need a forward method to the kalman filter

In [ ]:
#| export
from meteo_imp.kalman.filter import *
from torch.distributions import MultivariateNormal

In [ ]:
#| export
@patch
def _predict_filter(self: KalmanFilter, data, mask):
    """Predict every obsevation using only the filter step"""
    # use the predicted state not the filtered state!
    obs, mask = self._parse_obs(data, mask)
    pred_state_mean, pred_state_cov, _, _ = self._filter_all(obs, mask)
    mean, cov = self._obs_from_state(ListMNormal(pred_state_mean.squeeze(-1), pred_state_cov))
    
    return ListNormal(mean, cov2std(cov))

In [ ]:
#| export
@patch
def forward(self: KalmanFilter, masked_data: MaskedTensor):
    data, mask = masked_data
    assert not data.isnan().any()
    use_smooth = self.use_smooth if hasattr(self, 'use_smooth') else True
    
    mean, std = (self.predict(obs=data, mask=mask, smooth=True) if use_smooth
                        else self._predict_filter(data, mask))
    return NormalsParams(mean, std) # to have fastai working this needs to be a tuple subclass

In [ ]:
input = dls.one_batch()[0]
target = dls.one_batch()[1]

In [ ]:
model = KalmanFilter.init_simple(n_dim = hai.shape[-1])

In [ ]:
model.state_dict()

In [ ]:
data = input[0][0]
data.shape

In [ ]:
mask = input[1][0]

In [ ]:
mask.shape

In [ ]:
data.device

In [ ]:
torch.device

In [ ]:
data.shape, mask.shape

In [ ]:
model.predict(data, mask);

In [ ]:
model.use_smooth = True

In [ ]:
pred = model(input)

In [ ]:
pred[0].shape

In [ ]:
pred[1].shape

In [ ]:
model.use_smooth = False

In [ ]:
pred_filt = model(input)

In [ ]:
pred_filt[1].shape

In [ ]:
pred

In [ ]:
type(pred), type(pred_filt)

In [ ]:
test_ne(pred, pred_filt)

### Loss Function

add support for complete loss (also outside gap) and for filter loss (don't run the smooher)

There are two ways to compute the loss, one is to do it for all predictions the other is for doing it for only the gap
- only_gap

Play around with flatting + diagonal

In [ ]:
a = torch.diag(torch.tensor([1,2,3]))
d = torch.stack([a, a*10])
m = torch.stack([a.diag(), a.diag()*10])
d

In [ ]:
m.flatten()

In [ ]:
d

In [ ]:
torch.diagonal(d, dim1=1, dim2=2).flatten()

In [ ]:
means, stds = pred
data, mask = target

In [ ]:
# make a big matrix with all variables and observations and compute ll
mask = mask.flatten() 
obs = data.flatten()[mask]
means = data.flatten()[mask]
stds = stds.flatten()[mask] # need to support batches

MultivariateNormal(means, torch.diag(stds)).log_prob(obs)

In [ ]:
#| export
class KalmanLoss():
    def __init__(self,
                 only_gap:bool=True, # loss for all predictions or only gap
                ):
        store_attr()
    
    def __call__(self, pred: NormalsParams, target: MaskedTensor):
        data, mask = target
        means, stds = pred        
        assert not stds.isnan().any()
        loss = torch.zeros(1, device=data.device, dtype=data.dtype)
        for d, m, mean, std in zip(data, mask, means, stds):
            loss += self._loss_batch(d,m,mean, std)
        return loss
    
    def _loss_batch(self, data, mask, mean, std):
        # make a big vector with all variables and observations and compute ll
        mask = mask.flatten() if self.only_gap else torch.fill(mask, True).flatten()
        obs = data.flatten()[mask]
        mean = data.flatten()[mask]
        std = std.flatten()[mask] 
        
        return MultivariateNormal(mean, torch.diag(std)).log_prob(obs)
        

In [ ]:
means, stds = pred

In [ ]:
stds.shape

In [ ]:
means.shape

In [ ]:
data.isnan().any()

In [ ]:
mask.isnan().any()

In [ ]:
means.isnan().any()

In [ ]:
stds.isnan().sum()

In [ ]:
stds

In [ ]:
is_posdef_eigv(torch.diag(stds.flatten()))

In [ ]:
KalmanLoss(only_gap=True)(pred, target)

In [ ]:
KalmanLoss(only_gap=False)(pred, target)

### Metrics

Wrapper around fastai metrics to support masked tensors and normal distributions

In [ ]:
#| export
def to_meteo_imp_metric(metric):
    def meteo_imp_metric(inp, targ):
        return metric(imp[0], targ[0]) # first element are the means

### Callback

save the model state 

In [ ]:
#| export
from fastai.callback.all import *

In [ ]:
#| export
class SaveParams(Callback):
    def __init__(self, param_name):
        super().__init__()
        self.params = []
        self.param_name = param_name
    def after_batch(self):
        param = getattr(self.model, self.param_name).detach()
        self.params.append(param)

In [ ]:
#| export
class SaveParams(Callback):
    def __init__(self, param_name):
        super().__init__()
        self.params = []
        self.param_name = param_name
    def after_batch(self):
        param = getattr(self.model, self.param_name).detach()
        self.params.append(param)

### Learner

In [ ]:
#| export
from fastai.learner import * 

from fastai.tabular.all import *

from fastai.tabular.learner import *

from fastai.callback.progress import ShowGraphCallback

In [ ]:
obs_cov_history = SaveParams('obs_cov')

In [ ]:
all_data = CollectDataCallback()

In [ ]:
from fastai.metrics import RMSE

In [ ]:
model = KalmanFilter.init_random(n_dim_obs = hai.shape[1], n_dim_state = hai.shape[1]).cuda()

In [ ]:
model.use_smooth = False

In [ ]:
# model._set_constraint('obs_cov', model.obs_cov, train=False)

In [ ]:
dls = make_dataloader(hai[:20000], 200, 10, bs=10)

In [ ]:
input, target = dls.one_batch()
pred = model(input)
KalmanLoss()(pred, target)

In [ ]:
learn = Learner(dls, model, loss_func=KalmanLoss(only_gap=False), cbs = [] )

In [ ]:
learn.fit(1, 1e-3)

In [ ]:
learn.model.state_dict()

In [ ]:
from pyprojroot import here

In [ ]:
# torch.save(learn.model, here('trained_models'))

In [ ]:
learn.model.obs_cov

In [ ]:
learn.model.trans_cov_raw

In [ ]:
# torch.save(learn.model.state_dict(), "partial_traning_15_dec_not_pos_def_error")

In [ ]:
learn.recorder.plot_loss()

In [ ]:
# learn.lr_find()

In [ ]:
display_as_row(learn.model.get_info())

In [ ]:
learn.show_results()

## Float64

In [ ]:
model64 = KalmanFilter.init_random(hai.shape[1], hai.shape[1], dtype=torch.float64).cuda()

In [ ]:
dls64 = make_dataloader(hai64, 200, 10, bs=100) 

In [ ]:
input = dls64.one_batch()[0]
target = dls64.one_batch()[1]

In [ ]:
data, mask = input

In [ ]:
data.device

In [ ]:
data.dtype

In [ ]:
model64.predict(data);

In [ ]:
pred = model64(input)

In [ ]:
KalmanLoss()(pred, target)

In [ ]:
#| export
class Float64Callback(Callback):
    order = Recorder.order + 10 # run 
    def before_fit(self):
        self.recorder.smooth_loss.val = torch.tensor(0, dtype=torch.float64) # default is a float 32

In [ ]:
model64.use_smooth = False

In [ ]:
learn64 = Learner(dls64, model64, loss_func=KalmanLoss(), cbs = [Float64Callback] )

In [ ]:
learn64.fit(1, 1e-3)

In [ ]:
input[0].device

### Predictions

In [ ]:
learn64.predict(input)

In [ ]:
dls.show_results()

In [ ]:
learn64.show_results()